In [1]:
import sys
sys.path.append("../cs224u")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from collections import Counter
import vsm
from tqdm import tqdm

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
analysts = pd.read_csv('./data/analysts.csv')
print(analysts.shape)
analysts.head(10)

(187327, 12)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,analyst-company
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",Bruce Geller - DGHM
1,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",John Healy - Northcoast Research
2,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Kevin Steinke - Barrington Research Associates...
3,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Joe Box - KeyBanc Capital Markets
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET","Justin Hauke - Robert W. Baird & Company, Inc."
5,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Andrew Steinerman - JPMorgan
6,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Kevin McVeigh - Deutsche Bank
7,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Mark Marcon - Baird
8,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Greg Francfort - Bank of America
9,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Nicole Miller - Piper Jaffray


In [3]:
execs = pd.read_csv('./data/executives.csv')
print(execs.shape)
execs.head(10)

(111916, 12)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,executive-positions
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",Bob Stewart - President and Chief Financial Of...
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",Michael Weinstein - Chairman and Chief Executi...
2,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Ronald Croatti - President and CEO
3,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",Steven Sintros - SVP and CFO
4,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Alice Washington - Interim General Counsel
5,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Kate Duchene - Chief Executive Officer
6,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",Herb Mueller - Chief Financial Officer
7,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Cliff Hudson - CEO
8,4,Sonic Corp.,NASDAQ,SONC,2017-01-04 21:41:07,Sonic's (SONC) CEO Cliff Hudson on Q1 2017 Res...,Sonic Corp.,Q1 2017 Results Earnings Conference Call,NaN,SONC,"January 04, 2017 05:00 PM ET",Claudia San Pedro - CFO
9,5,"Walgreens Boots Alliance, Inc.",NASDAQ,WBA,2017-01-05 16:33:05,Walgreens Boots Alliance's (WBA) CEO Stefano P...,"Walgreens Boots Alliance, Inc.",Q1 2017 Results Earnings Conference Call,NaN,WBA,"January 05, 2017, 08:30 AM ET","Gerald Gradwell - Senior Vice President, Inves..."


In [4]:
qna = pd.read_csv('./data/qna.csv')
print(qna.shape)
qna.head(10)

(1828257, 14)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Operator,[Operator Instructions] Our first question com...
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Bruce Geller,Hi Good morning guys.
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Hi Bruce, how are you?"
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Bruce Geller,Good thanks. Glad to hear things have turned a...
4,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",5,Bob Stewart,"Yes, that includes an additional week last yea..."
5,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",6,Bruce Geller,So it's not nearly as...
6,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",7,Bob Stewart,"No, no."
7,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",8,Bruce Howard Geller,…as dismal as it might imply. What were the ap...
8,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",9,Bob Stewart,"About 2%. Probably down, somewhere along that ..."
9,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",10,Bruce Geller,Got it. Okay. Can you just kind of go through ...


In [5]:
statements = pd.read_csv('./data/statements.csv')
print(statements.shape)
statements.head(10)

(204656, 14)


,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Presentation,NaN
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Operator,"Greetings, and welcome to the Ark Restaurants ..."
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Thank you, operator. Good morning, and thank y..."
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Michael Weinstein,"Hi, everybody. If you read the release, we're ..."
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",1,Presentation,NaN
5,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",2,Operator,"Ladies and gentlemen, thank you for standing b..."
6,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",3,Steven Sintros,Thank you and welcome to the UniFirst Corporat...
7,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",4,Ronald Croatti,"Thank you, Steve. I'd like to welcome everyone..."
8,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",5,Steven Sintros,"Thanks, Ron. As Ron mentioned, revenues for th..."
9,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",1,Presentation,NaN


In [142]:
row_list = []

statement_idx, qna_idx = 0, 0
consecutive_no_append = 0
while(statement_idx < statements.shape[0] - 1 and qna_idx <= 155051):
    statement_company_name = statements.iloc[statement_idx]['file-company-name']
    qna_company_name = qna.iloc[qna_idx]['file-company-name']
    should_append = statement_company_name == qna_company_name  # If they don't match then this statement doesn't have Q&A session
    print(should_append)
    print(statement_idx)
    print(qna_idx)
    print("")
    if not should_append: consecutive_no_append += 1
    stop_statements = False
    # Only increment Q&A pointer if we're appending statement rows to row_list
    stop_qna = (qna_idx == (qna.shape[0] - 1)) or (not should_append)
    while(True):
        if should_append and not stop_statements: row_list.append(statements.iloc[statement_idx])
        
        # Always increment statements pointer
        if not stop_statements:
            curr_num = statements.iloc[statement_idx]['num']
            statement_idx += 1
            next_num = statements.iloc[statement_idx]['num']
            if curr_num != next_num or statement_idx == (statements.shape[0] - 1):
                stop_statements = True
        
        if not stop_qna:
            curr_num = qna.iloc[qna_idx]['num']
            qna_idx += 1
            next_num = qna.iloc[qna_idx]['num']
            if curr_num != next_num or qna_idx == (qna.shape[0] - 1):
                stop_qna = True
        
        if stop_statements and stop_qna:
            break
            
cols = ['num', 'file-company-name', 'file-exchange', 'file-symbol', 'file-published-on', 'file-title', 'company-name', 'quarter-year-event', 'symbol-exchange', 'symbol', 'datetime', 'order', 'name', 'content']
comp_statements = pd.DataFrame(row_list,columns = cols)
print(comp_statements.shape)
comp_statements.head(10)


True
0
0

True
4
51

True
9
99

True
15
144

True
19
210

True
25
287

True
31
363

True
37
380

True
45
424

True
50
530

True
58
604

True
63
747

True
69
784

True
74
873

True
80
903

True
85
947

True
91
977

True
97
1027

True
102
1094

True
110
1147

True
115
1243

True
120
1308

True
126
1387

True
132
1452

True
137
1535

True
141
1624

True
146
1681

True
153
1745

True
159
1811

True
166
1894

False
172
1928

True
175
1928

False
180
2019

True
183
2019

True
190
2036

True
195
2067

True
199
2138

True
205
2178

True
226
2182

True
229
2245

True
235
2283

False
244
2400

True
247
2400

True
249
2460

True
255
2505

True
257
2549

True
263
2619

True
268
2649

False
271
2743

True
274
2743

True
278
2837

True
281
2892

True
287
2989

True
294
3009

True
300
3083

True
305
3118

True
310
3143

True
315
3185

True
318
3234

True
321
3298

True
327
3356

True
331
3387

True
338
3536

True
345
3610

False
348
3614

True
350
3614

True
355
3678

True
360
3709

True
365
3834

Tr

True
2897
36475

True
2901
36530

True
2904
36623

True
2912
36693

True
2915
36753

True
2921
36757

True
2925
36781

True
2932
36864

True
2937
36979

True
2940
37016

False
2946
37094

True
2952
37094

True
2955
37119

True
2959
37168

True
2970
37229

True
2975
37269

True
2982
37329

True
2989
37391

True
2995
37445

True
3000
37455

True
3005
37526

True
3011
37703

True
3016
37741

True
3021
37778

True
3028
37924

True
3033
37987

True
3039
38084

True
3044
38168

True
3052
38250

True
3059
38333

True
3066
38403

True
3070
38419

True
3076
38472

True
3080
38511

True
3085
38595

True
3090
38698

True
3096
38755

True
3103
38806

True
3109
38853

True
3115
38946

True
3121
38993

True
3126
39047

True
3132
39115

True
3138
39194

True
3144
39239

True
3150
39342

True
3156
39374

True
3162
39433

True
3169
39469

True
3175
39527

True
3184
39590

True
3188
39650

True
3193
39741

True
3199
39788

True
3204
39814

True
3210
39879

True
3213
39929

True
3220
39984

True
3223
400

True
5639
70242

True
5646
70323

True
5650
70395

True
5655
70470

True
5659
70592

True
5667
70626

True
5673
70746

True
5678
70867

True
5685
70944

True
5688
71088

True
5695
71181

True
5701
71283

True
5704
71379

True
5713
71429

True
5718
71503

True
5724
71534

True
5730
71636

True
5738
71687

True
5743
71790

True
5749
71852

True
5754
71949

True
5759
72124

True
5764
72213

True
5769
72399

True
5775
72469

True
5781
72494

True
5787
72511

True
5791
72611

True
5796
72733

True
5801
72774

True
5808
72824

True
5813
72915

True
5819
73015

True
5826
73075

True
5832
73151

True
5840
73254

True
5846
73329

True
5851
73413

True
5858
73540

True
5864
73598

True
5872
73674

True
5877
73729

True
5885
73779

True
5892
73824

True
5895
73875

True
5900
73916

True
5904
74044

True
5910
74124

True
5916
74179

True
5921
74233

True
5926
74370

True
5929
74467

True
5934
74556

True
5939
74677

True
5946
74686

True
5952
74795

True
5958
74926

True
5964
75015

True
5970
7513

True
8364
101878

True
8371
101914

True
8378
101945

False
8385
101987

True
8387
101987

True
8392
102133

True
8397
102188

True
8402
102194

True
8410
102248

True
8416
102293

True
8426
102361

True
8428
102387

True
8431
102419

True
8435
102453

True
8440
102503

True
8446
102599

True
8449
102619

True
8453
102710

True
8457
102712

True
8462
102797

True
8470
102838

True
8476
102875

True
8480
102903

True
8484
102989

True
8492
103025

True
8497
103068

True
8501
103133

True
8506
103175

True
8510
103212

True
8514
103236

False
8520
103256

True
8524
103256

True
8529
103267

True
8536
103285

True
8540
103286

True
8546
103415

True
8552
103486

True
8556
103587

True
8562
103686

False
8576
103747

True
8580
103747

True
8586
103832

True
8592
103917

True
8598
103953

True
8604
104071

True
8610
104073

True
8615
104162

True
8620
104240

True
8625
104322

True
8632
104374

True
8638
104429

True
8643
104481

True
8647
104555

True
8651
104580

True
8658
104609

True
86

True
10981
131225

True
10984
131345

True
10991
131423

True
10996
131479

True
11002
131574

True
11008
131657

True
11014
131699

True
11019
131773

True
11025
131879

True
11030
131923

True
11033
131941

True
11039
132009

True
11044
132108

True
11050
132155

True
11055
132189

True
11061
132347

True
11065
132447

True
11071
132557

True
11077
132579

True
11084
132636

True
11090
132664

True
11096
132762

True
11104
132860

True
11110
132930

True
11115
133062

True
11120
133106

True
11126
133235

True
11130
133343

True
11133
133428

True
11139
133502

True
11145
133591

True
11150
133709

True
11154
133770

True
11159
133817

True
11166
133903

True
11171
133932

True
11177
133974

True
11183
134016

True
11189
134062

True
11195
134157

True
11203
134183

True
11209
134231

True
11212
134249

True
11217
134260

True
11221
134320

True
11229
134363

True
11234
134398

True
11238
134474

True
11242
134538

True
11247
134638

True
11253
134696

True
11258
134740

True
11263
1

,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Presentation,NaN
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Operator,"Greetings, and welcome to the Ark Restaurants ..."
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Thank you, operator. Good morning, and thank y..."
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Michael Weinstein,"Hi, everybody. If you read the release, we're ..."
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",1,Presentation,NaN
5,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",2,Operator,"Ladies and gentlemen, thank you for standing b..."
6,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",3,Steven Sintros,Thank you and welcome to the UniFirst Corporat...
7,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",4,Ronald Croatti,"Thank you, Steve. I'd like to welcome everyone..."
8,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",5,Steven Sintros,"Thanks, Ron. As Ron mentioned, revenues for th..."
9,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",1,Presentation,NaN


In [145]:
qna_mini = qna.iloc[:155052]
qna_mini.to_pickle("qna_mini")

In [146]:
comp_statements.to_pickle("statements_mini")

In [154]:
test_qna_mini = pd.read_pickle('qna_mini')
test_qna_mini.head(120)

,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Operator,[Operator Instructions] Our first question com...
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Bruce Geller,Hi Good morning guys.
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Hi Bruce, how are you?"
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Bruce Geller,Good thanks. Glad to hear things have turned a...
4,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",5,Bob Stewart,"Yes, that includes an additional week last yea..."
5,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",6,Bruce Geller,So it's not nearly as...
6,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",7,Bob Stewart,"No, no."
7,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",8,Bruce Howard Geller,…as dismal as it might imply. What were the ap...
8,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",9,Bob Stewart,"About 2%. Probably down, somewhere along that ..."
9,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",10,Bruce Geller,Got it. Okay. Can you just kind of go through ...


In [149]:
test_statements_mini = pd.read_pickle('statements_mini')
test_statements_mini.head(50)

,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Presentation,NaN
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Operator,"Greetings, and welcome to the Ark Restaurants ..."
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Thank you, operator. Good morning, and thank y..."
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Michael Weinstein,"Hi, everybody. If you read the release, we're ..."
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",1,Presentation,NaN
5,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",2,Operator,"Ladies and gentlemen, thank you for standing b..."
6,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",3,Steven Sintros,Thank you and welcome to the UniFirst Corporat...
7,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",4,Ronald Croatti,"Thank you, Steve. I'd like to welcome everyone..."
8,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",5,Steven Sintros,"Thanks, Ron. As Ron mentioned, revenues for th..."
9,3,"Resources Connection, Inc.",NASDAQ,RECN,2017-01-04 23:16:06,Resources Connection's (RECN) CEO Kate Duchene...,"Resources Connection, Inc.",Q2 2017 Earnings Conference Call,NaN,RECN,"January 4, 2017 17:00 ET",1,Presentation,NaN


Slower version of cleaning data but better at dealing with Q&A sessions with no corresponding statement

In [ ]:
row_list = []
for i in range(statements.shape[0]):
    publish_date = statements.iloc[i]['file-published-on']
    if len(qna.loc[qna['file-published-on'] == publish_date]) != 0: row_list.append(statements.iloc[i])

cols = ['num', 'file-company-name', 'file-exchange', 'file-symbol', 'file-published-on', 'file-title', 'company-name', 'quarter-year-event', 'symbol-exchange', 'symbol', 'datetime', 'order', 'name', 'content']
comp_statements = pd.DataFrame(row_list,columns = cols)
print(comp_statements.shape)
comp_statements.head(10)
comp_statements.to_csv('comp_statements’)

In [143]:
row_list = []
for i in range(execs.shape[0]):
    publish_date = execs.iloc[i]['file-published-on']
    if len(qna.loc[qna['file-published-on'] == publish_date]) != 0: row_list.append(execs.iloc[i])

cols = ['num', 'file-company-name', 'file-exchange', 'file-symbol', 'file-published-on', 'file-title', 'company-name', 'quarter-year-event', 'symbol-exchange', 'symbol', 'datetime', 'executive-positions']
comp_execs = pd.DataFrame(row_list,columns = cols)
print(comp_execs.shape)
comp_execs.head(10)
comp_execs.to_csv('comp_execs')

KeyboardInterrupt: 